In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/keyframe-8'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/keyframe-8/L08_V023/L08_V023.json
/kaggle/input/keyframe-8/L08_V023/keyframes/208.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/473.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/333.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/45.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/369.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/56.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/89.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/20.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/275.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/212.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/239.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/58.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/150.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/6.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/109.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/149.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/187.jpg
/kaggle/input/keyframe-8/L08_V023/keyframes/436.jpg
/kaggle/input/keyframe-

In [2]:
!pip install ultralytics
!pip install -q git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.1/872.1 kB 12.6 MB/s eta 0:00:00a 0:00:01


In [3]:
!pip install colorthief

In [4]:
!pip install cloudinary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 3.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cloudinary: filename=cloudinary-1.41.0-py3-none-any.whl size=139738 sha256=c6322d8bcd64afb9318c3741558be6aa70817ddd56a552e373277a3c541e060e
  Stored in directory: /root/.cache/pip/wheels/4a/55/5f/756f34a12b7e3c2923b8dbe52d02299d23bd91a50948fe0929
Successfully built cloudinary


In [5]:
import os
import json
import torch
import clip
from PIL import Image
from io import BytesIO
from ultralytics import YOLO
from colorthief import ColorThief
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import cloudinary.uploader, cloudinary

class ImageProcessor:
    def __init__(self, basic_colors, clip_model='ViT-B/32', yolo_model_path='yolov8m.pt', device='cuda'):
        self.device = device
        self.clip_model, self.clip_preprocess = clip.load(clip_model, device=device)
        self.yolo_model = YOLO(yolo_model_path)
        self.yolo_model.to(self.device)  # Ensure YOLO model is on GPU
        self.basic_colors = basic_colors
        self.cloud_name = "dflfckocl"
        self.api_key = "983512829559384"
        self.api_secret = "y9dA-RfVpHvn6MS6yPYd9KjYJw0"

        # Configure Cloudinary
        cloudinary.config(
            cloud_name=self.cloud_name,
            api_key=self.api_key,
            api_secret=self.api_secret
        )

    def read_json(self, file_path):
        with open(file_path, 'r') as file:
            return json.load(file)
        
    def save_json(self, file_path, data):
        # Ensure the directory exists
        directory = os.path.dirname(file_path)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        # Save the JSON file
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)
        print(f'Successfully saved {file_path}')

    def downscale_image(self, image):
        return image.resize((256, 144))
    
    def extract_clip_features(self, image):
        image_input = self.clip_preprocess(image).unsqueeze(0).to(self.device)
        with torch.no_grad():
            image_features = self.clip_model.encode_image(image_input)
        return image_features.cpu().numpy().flatten()

    def detect_objects(self, image_path):
        results = self.yolo_model.predict(image_path, conf=0.25, verbose=False)
        object_counts = {}
        for result in results:
            for box in result.boxes:
                label = result.names[box.cls[0].item()]
                object_counts[label] = object_counts.get(label, 0) + 1
        return object_counts
    
    def get_main_colors(self, image_path, num_colors=3):
        color_thief = ColorThief(image_path)
        palette = color_thief.get_palette(color_count=num_colors)
        return palette

    def get_closest_color(self, rgb_color):
        closest_color = None
        min_distance = float("inf")
        for color_name, color_value in self.basic_colors.items():
            distance = np.sqrt(
                (rgb_color[0] - color_value[0]) ** 2 +
                (rgb_color[1] - color_value[1]) ** 2 +
                (rgb_color[2] - color_value[2]) ** 2
            )
            if distance < min_distance:
                min_distance = distance
                closest_color = color_name
        return closest_color

    def process_image(self, image_path, frame_index):
        image = Image.open(image_path)

        vector_feature = self.extract_clip_features(image)

        label_counts = self.detect_objects(image_path)

        main_colors_rgb = self.get_main_colors(image_path)

        main_color_names = [self.get_closest_color(color) for color in main_colors_rgb]

        downscaled_image = self.downscale_image(image)

        # Upload image to Cloudinary
        buffer = BytesIO()
        downscaled_image.save(buffer, format='JPEG')
        buffer.seek(0)
        
        try:
            response = cloudinary.uploader.upload(buffer.getvalue(), folder='aichallenge')
            url = response['secure_url']
        except Exception as e:
            print(f"Error uploading to Cloudinary: {e}")
            url = f"https://example.com/default.jpg"

        return {
            "url": url,
            "vector_feature": vector_feature.tolist(),
            "frame_index": frame_index,
            "yolo": label_counts,
            "main_color": main_color_names
        }

    def process_images_in_subfolder(self, base_folder, batch_size=10):
        for subfolder in os.listdir(base_folder):
            objectName = subfolder.split('_')[1]
            subfolder_path = os.path.join(base_folder, subfolder)
            if os.path.isdir(subfolder_path):
                mapping_path = os.path.join(subfolder_path, f'{subfolder}.json')
                mapping_file = self.read_json(mapping_path)

                subfolder_data = {}
                keyframes_folder = os.path.join(subfolder_path, 'keyframes')

                # Process images in batches
                keys = list(mapping_file.keys())
                for i in range(0, len(keys), batch_size):
                    batch_keys = keys[i:i+batch_size]
                    with ThreadPoolExecutor() as executor:
                        futures = []
                        for index in batch_keys:
                            frame_index = mapping_file[index]
                            image_path = os.path.join(keyframes_folder, f'{index}.jpg')
                            futures.append(executor.submit(self.process_image, image_path, frame_index))

                        for future in as_completed(futures):
                            result = future.result()
                            subfolder_data[result["frame_index"]] = result

                output_path = f'/kaggle/working/output/{subfolder}_details.json'
                self.save_json(output_path, subfolder_data)

basic_colors = {
    "white": (255, 255, 255),
    "red": (255, 0, 0),
    "yellow": (255, 255, 0),
    "orange": (255, 165, 0),
    "green": (0, 128, 0),
    "cyan": (0, 255, 255),
    "blue": (0, 0, 255),
    "purple": (128, 0, 128),
    "black": (0, 0, 0)
}

if __name__ == '__main__':
    imageProcessor = ImageProcessor(basic_colors=basic_colors, yolo_model_path='yolov8m.pt')
    imageProcessor.process_images_in_subfolder('/kaggle/input/keyframe-8', batch_size=10)


100%|███████████████████████████████████████| 338M/338M [00:09<00:00, 36.9MiB/s]


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
"""import os
import json
import torch
import clip
from PIL import Image
from ultralytics import YOLO
from colorthief import ColorThief
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np

class ImageProcessor:
    def __init__(self, basic_colors, clip_model='ViT-B/32', yolo_model_path='yolov8m.pt', device='cuda'):
        self.device = device
        self.clip_model, self.clip_preprocess = clip.load(clip_model, device=device)
        self.yolo_model = YOLO(yolo_model_path)
        self.yolo_model.to(self.device)  # Ensure YOLO model is on GPU
        self.basic_colors = basic_colors

    def read_json(self, file_path):
        with open(file_path, 'r') as file:
            return json.load(file)
        
    def save_json(self, file_path, data):
        # Ensure the directory exists
        directory = os.path.dirname(file_path)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        # Save the JSON file
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)
        print(f'Successfully saved {file_path}')

    def downscale_image(self, image):
        return image.resize((256, 144))
    
    def extract_clip_features(self, image):
        image_input = self.clip_preprocess(image).unsqueeze(0).to(self.device)
        with torch.no_grad():
            image_features = self.clip_model.encode_image(image_input)
        return image_features.cpu().numpy().flatten()

    def detect_objects(self, image_path):
        results = self.yolo_model.predict(image_path, conf=0.25, verbose=False)
        object_counts = {}
        for result in results:
            for box in result.boxes:
                label = result.names[box.cls[0].item()]
                object_counts[label] = object_counts.get(label, 0) + 1
        return object_counts
    
    def get_main_colors(self, image_path, num_colors=3):
        color_thief = ColorThief(image_path)
        palette = color_thief.get_palette(color_count=num_colors)
        return palette

    def get_closest_color(self, rgb_color):
        closest_color = None
        min_distance = float("inf")
        for color_name, color_value in self.basic_colors.items():
            distance = np.sqrt(
                (rgb_color[0] - color_value[0]) ** 2 +
                (rgb_color[1] - color_value[1]) ** 2 +
                (rgb_color[2] - color_value[2]) ** 2
            )
            if distance < min_distance:
                min_distance = distance
                closest_color = color_name
        return closest_color

    def process_image(self, image_path, frame_index):
        image = Image.open(image_path)

        vector_feature = self.extract_clip_features(image)

        label_counts = self.detect_objects(image_path)

        main_colors_rgb = self.get_main_colors(image_path)

        main_color_names = [self.get_closest_color(color) for color in main_colors_rgb]

        downscaled_image = self.downscale_image(image)

        return {
            "vector_feature": vector_feature.tolist(),
            "frame_index": frame_index,
            "yolo": label_counts,
            "main_color": main_color_names
        }

    def process_images_in_subfolder(self, base_folder, batch_size=10):
        for subfolder in os.listdir(base_folder):
            objectName = subfolder.split('_')[1]
            subfolder_path = os.path.join(base_folder, subfolder)
            if os.path.isdir(subfolder_path):
                mapping_path = os.path.join(subfolder_path, f'{subfolder}.json')
                mapping_file = self.read_json(mapping_path)

                subfolder_data = {}
                keyframes_folder = os.path.join(subfolder_path, 'keyframes')

                # Process images in batches
                keys = list(mapping_file.keys())
                for i in range(0, len(keys), batch_size):
                    batch_keys = keys[i:i+batch_size]
                    with ThreadPoolExecutor() as executor:
                        futures = []
                        for index in batch_keys:
                            frame_index = mapping_file[index]
                            image_path = os.path.join(keyframes_folder, f'{index}.jpg')
                            futures.append(executor.submit(self.process_image, image_path, frame_index))

                        for future in as_completed(futures):
                            result = future.result()
                            subfolder_data[result["frame_index"]] = result

                output_path = f'/kaggle/working/output/{subfolder}_details.json'
                self.save_json(output_path, subfolder_data)

basic_colors = {
    "white": (255, 255, 255),
    "red": (255, 0, 0),
    "yellow": (255, 255, 0),
    "orange": (255, 165, 0),
    "green": (0, 128, 0),
    "cyan": (0, 255, 255),
    "blue": (0, 0, 255),
    "purple": (128, 0, 128),
    "black": (0, 0, 0)
}

if __name__ == '__main__':
    imageProcessor = ImageProcessor(basic_colors=basic_colors, yolo_model_path='yolov8m.pt')
    imageProcessor.process_images_in_subfolder('/kaggle/input/keyframe-10', batch_size=10)
"""
#Thís using for multiprocessor
